In [ ]:
clc, clear all
cat ~/.jupyter/octave_kernel_config.py
# use Qt as the default backend for plots
c.OctaveKernel.plot_settings = dict(backend='qt')

# Energy Management in Building

## 0. Introduction

In this report, we present the results that we obtained after analyzing the thermical behavior of a building. The aim of this exercice is to create a virtual model representing our building in order to run the thermical simulation with a simplified model. 

First, we are going to describe our building, its location, geometry, use and materials. We will then define the hypothesis used to make the simplified model. After that, we will define the conditions needed to run the model, such as the local weather, the internal sources (occupants, electronic devices, lightings) and the heating/ventilation system. 

In a second part, we are going to draw the diagram that we will use to defined the terms of the different matrices needed to run the calculations. This diagram is supposed to schematize the thermical behaviour of the building. We will do it with different steps : each one will add a difficulty to the model, such as a heating and ventilation system and the orientation and the direction of the sun rays. We will also explain why we made different choices to modelise/discretise some materials compared to others. 

We used a Matlab program to run the calculations, the code will be presented all along this report in order to be able to run it here directly.

In a last part, we will implemant the model in DesignBuilder in order to compare the results with our calculations from Matlab (Temperature differences between inside and outside, and power needed to maintain an acceptable temperature inside the building). We will also try to change some parameters in order to improve the building energy consumption, such as different heating system, changing the thickness of the materials or adding some solar protections.


## 1. Building Description

### 1.1 Location

The building we are going to work on is located in Poitiers, in the center of France.

<img src="dijon.png" alt="drawing" width="400"/>

### 1.2 Space and geometry
We decided to work on a workspace, with a room with two desks (25,9 m2) and a kitchen (10,1 m2). It is a rectangular plan with a large glass wall (1,2 cm of argon between two 0,4 cm glasses) facing North and 3 insulated brick walls (22 cm of bricks, 18 cm of wood wool insulation and 10,5 cm of exterior brick cladding). There is also one wood door (5 cm) to the outside, one wood door (5 cm) in the partition wall and an other window with the same thickness than the glass wall. The partition wall is composed of plaster with no insulation (5,4 cm of air between two BA13). 
Here is the plan : 

<img src="PLAN_projet.png" alt="drawing" width="800"/>

We are supposed, in a second part, to modelise our workspace on DesignBuilder in order to compare the results. The height of the ceiling is 2,5 m. 
Here is the 3D model obtained :

<img src="Vue 3D.png" alt="drawing" width="800"/>

We considered the workspace to be on the ground floor, with sufficient insulation in order to consider there is no flow passing through the floor. We also consider that there is the same workspace on the 1st floor of the building, with the same caracteristics and inside temperature than ours, so there is also no flow between our workspace and the upper floor.

### 1.3 Materials' properties and others needed values


Here is the list of the different materials we chose to use in this project, with their thickness and caracteristics : 


| Material | Thickness (m) | Thermal conductivity (W/mK | Density (kg/m3) | Specific heat (J/(kg.K)) |
| ----------- | ----------- | ----------- | ----------- | ----------- | 
| Structural brick wall | 0,22 | 1,310 | 1700 | 707 | 
| Cladding (bricks) | 0,105 | 1,310 | 1700 | 707 | 
| Wood wool insulation | 0,18 | 0,036 | 55 | 2100 | 
| Plaster | 0,013 | 0,35 | 950 | 1088 | 
| Glass | 0,004 | 1,05 | 2500 | 840 | 
| Argon | 0,012 | 0,018 | 1,378 | 520 | 
| Air | 0,054 | 0,024 | 1,225 | 1004 | 
| Wood door | 0,05 | 0,120 | 500 | 2301 |



The surfaces and volumes are given in the next tables : 


| Type of panel | Surface (m2) |
| ----------- | ----------- |
| Total surface of office's wall | 23,20 |
| Total surface of kitchen's wall | 15,40 |
| Surface of glass wall in the office part | 15,95 |
| Surface of glass wall in the kitchen part | 6,40 |
| Surface of the window | 1,50 |
| Surface of one door | 1,80 |
| Surface of one plaster board | 8,20 |



| Type of space | Volume (m3) |
| ----------- | ----------- |
| Air of the office | 23,20 |
| Air of the kitchen | 15,40 |




The coefficient convection are the following : 

ho = 10 W/(m2.K) for the outside

hi = 10 W/(m2.K) for the inside




In [ ]:
% width (m)
w_brick_in = 0.22;
w_brick_out = 0.105;
w_insulation = 0.18;
w_door = 0.05;
w_plaster = 0.013;
w_glass = 0.004;
w_argon = 0.012;
w_air = 0.054;

% Surfaces (en m2)
S_wall = 35.125;
S_wall_office = 2.5*(4+0.5+1.1+2.9)+1.5+0.5*0.9;
S_wall_kitchen = 2.5*(0.96+0.6+4)+1.5;
S_glasswall_kitchen = 6.4;
S_glasswall_office = 15.95;
S_window = 1.5;
S_door = 2*0.9;
S_plaster = 8.2;

V_air_kitchen = 2.5*10;
V_air_office = 2.5*25;

% Thermal conductivity (en W/mK)
l_brick = 1.310;
l_insulation = 0.036;
l_glass = 1.050;
l_wood =  0.120;
l_plaster = 0.35;
l_argon = 0.01772;
l_air = 0.024;

% specific heat (J/(kg.K))
cp_brick = 707;
cp_insulation = 2100;
cp_wood = 2301;
cp_glass = 840;
cp_air = 1004;
cp_argon = 520;
cp_plaster = 1088;

% density (kg/m3)
d_brick = 1700;
d_insulation = 55;
d_wood = 500;
d_plaster = 950;
d_glass = 2500;
d_air = 1.225; 
d_argon = 1.378;

% convection coefficients[W/m2 K]
ho = 10;    hi = 4;

%% Thermal capacity  (J/K)
C_brick_in_kitchen = d_brick*cp_brick*w_brick_in*S_wall_kitchen;
C_brick_out_kitchen = d_brick*cp_brick*w_brick_out*S_wall_kitchen;
C_brick_in_office = d_brick*cp_brick*w_brick_in*S_wall_office;
C_brick_out_office = d_brick*cp_brick*w_brick_out*S_wall_office;
C_insulation_office = d_insulation*cp_insulation*w_insulation*S_wall_office;
C_insulation_kitchen = d_insulation*cp_insulation*w_insulation*S_wall_kitchen;
C_wood = d_wood*cp_wood*w_door*S_door;
C_glass = d_glass*cp_glass*w_glass*S_window;
C_glasswall_kitchen = d_glass*cp_glass*w_glass*S_glasswall_kitchen;
C_glasswall_office = d_glass*cp_glass*w_glass*S_glasswall_office;
C_plaster = d_plaster*cp_plaster*S_plaster*w_plaster;
C_argon_glasswall_kitchen = d_argon*cp_argon*w_argon*S_glasswall_kitchen;
C_argon_glasswall_office = d_argon*cp_argon*w_argon*S_glasswall_office;
C_argon_window = d_argon*cp_argon*w_argon*S_window;
C_air_plaster = d_air*cp_air*S_plaster*0.54;

C_air_kitchen = d_air*cp_air*V_air_kitchen;
C_air_office = d_air*cp_air*V_air_office;

## 2. General hypothesis for modeling

1. We only have 1D thermal Exchange, because the thickness of the wall is small compared to the height and the width of the space.
2. We use the inside surface of the walls instead of the outside ones to neglect the thermical bridges.
3. All the external walls have the same composition and same characteristics.
4. There is no flow between the ground, the upper floor and our workspace.
5. There is no conduction at the junction between walls and glass, because the variation is negligible.
6. The density and the capacity of the air are identical in each room.
7. The flow absorbed by the window is not neglected.
8. We neglect reflected radiations.
9. The initial inside temperature is always 20°C.
10. Direct shortwaves are uniformly dispatched on the vertical surfaces only, proportionally to their surface.
11. We neglect the surface of the window and entrance door for the shape factor calculations of the walls. 
12. We neglect the thermal bridges.


## 3. Conditions to run the model



Intermediary seasons will not be modelled, in order to simplify the coding process. We estimate that it is not relevant for a study using Matlab programming. Indeed, our aim is not to precisely calculate heat losses on a real building, but rather to determinate and evaluate ways to optimise a design. Consumptions will be calculated on 100 days of summer (starting from the 21st of June) and 100 days of winter (starting from the 1st of January), and then averaged and annualized.

### 3.1 Local weather

Our building is located in Dijon : we will use the weather data sheet corresponding. We will collect all the input contained in this file with the function fReadWeather, so we will be able to get the changing values of outside temperature and radiation depending on the time.

In [ ]:
Hiver = 0 ;
if Hiver == 1
    period = 31*24;
    from =  1;
else
    period = 31*24;
    from = 6*30*24;  
end

[Time,Temp,RadNDir,RadHDif,WDir,WSpeed,month,day,hour,minute] = fReadWeather('FRA_DIJON_IWEC.csv',from,period);

### 3.2 Internal sources

We defined that the internal sources were composed of the heating coming from the electronic devices, the occupants and lightings. 

### 3.3 Heating and ventilation system

In the first step of the program, we will not consider a heating and ventilation system since we just want to verify that the program works. We will then implement a radiator and simulate a natural ventilation in order for the model to be more accurate.

## 4. Diagram

### 4.1 Model

We have two areas : the work space and the kitchen, that are separated by a partition wall. 


<img src="Diagramme.png" alt="drawing" width="1200"/>

### 4.2 Precisions on discretisation



## 5. 1st Simulation

The simulation will be done thanks to the model we draw in the previous chapter. We name 𝜃 the temperature of the nodes, e the temperature differences over resistances, q the heat flow through resistances and f the external fluxes.

In this part, we will not consider the orientation in the building. This caracteristic will be implemented il the following part. 

To solve the system, we had to solve the following equativo, which can be linked whith the previous pattern.

<img src="equation.png" alt="drawing" width="300"/>


The C-matrix respresents the capacity of the different surfaces. 
The A-matrix represents how a flow is connected to nodes. 
The G-matrix represents the conductances of the surfaces (conduction or convection). 
The b-matrix respresents the temperatures imposed in the system. 
The f-matrix represents the heat flow sources imposed in the system. 
 
 The matrix A, C and G are not time depending, we are gonna test and verify the coherence of the results of the Matlab code in static state before implementing the matrix f and b.


### 5.1 Definition of the matrix A, C and G

#### 5.1.1 Linking matrix A

We use the model presented before to create the linking matrix A. 
Beforehand we’ve decided the direction of each flow from each branch (figure ?). If the path from the branch to the node is in the same direction as the flow, the corresponding coefficient in the A-matrix will be 1. If the flow is in the opposite direction, the coefficient will be -1. We used for loops in order to modelise repetitive logical schemes (for the discretisation of a wall for example).

In [ ]:
A = zeros(58,51);
A(1,1) = 1;
for i = (2:10) 
    A(i,[i-1 i])= [-1,1];
end
for i = (11:15)
    A(i,[i i-1])= [-1,1];
end
A(16,15) = 1;
A(17,[10 16])= [1 -1];
for i =(18:19)
    A(i,[i-1 i-2])= [-1,1];
end 
A(20,18) = 1;

A(22,[10 19])= [1 -1];
for i = (23:26)
    A(i,[i-4 i-3]) = [1 -1];
end
A(27,23)=1;
A(28,[10 24])=[-1,1];
A(29,[24 25])=[-1 1];
A(30,[51 26])=[-1 1];
A(31,[26 30]) = [-1 1];
A(32,[10 27])=[-1 1];
for i= (33:35)
    A(i,[i-6 i-5]) = [-1 1];
end
A(36,31)=1;
for i=(37:44)
    A(i,[i-6 i-5])= [-1 1];
end
A(45,[39 30])=[-1 1];
A(46,40)=1;
for i = (47:50)
    A(i,[i-7 i-6]) = [-1 1];
end
A(51,[44 30])= [-1 1];
A(52,45)=1;
for i = (53:56)
    A(i,[i-8 i-7])= [-1 1];
end
A(57,[49 30])=[-1 1];

% partition wall error corection
A(21,[25 50])=[-1 1];
A(58,[50 51])=[-1 1];

#### 5.1.2 Capacities matrix C

We then use the values calculated before to create the capacities matrix. This matrice is defined depending on our choices of dimensions and materials. : 

In [ ]:
%% Capacities matrix
C=zeros(1,51);

% office
% wall office
C(2)=C_brick_out_office;
C(7)=C_brick_in_office ;C(8)=C(7);
C(4)= C_insulation_office; C(5)=C(4);
C(10)= C_air_office;

% glass wall office
C(12) = C_glasswall_office ; C(14)=C(12);
C(13)= C_argon_glasswall_office;

% office door
C(17)= C_wood;

% window office
C(20)= C_glass ; C(22)=C(20);
C(21)= C_argon_window;

% bond
% partition wall
C(25)= C_plaster ; C(51)=C(25);
C(50)= C_air_plaster;

% door
C(28)=C_wood;

% kitchen
% wall kitchen
C(32)=C_brick_out_kitchen;
C(37)=C_brick_in_kitchen ; C(38)=C(37);
C(34)= C_insulation_kitchen ; C(35)=C(34);
C(30)= C_air_kitchen;

% glass wall kitchen
C(41) = C_glasswall_office ; C(43)=C(41);
C(42)= C_argon_glasswall_office;


% window kitchen
C(46)= C_glass ; C(48)=C(46);
C(47)= C_argon_window;

C = diag(C);

#### 5.1.3 Conductance matrix G

The conductance is the reverse of the resistance for each surface. This matrice is also defined by our choices of dimensions and materials.

In [ ]:
% Resistances 
% outside wall office
R1 = 1/(ho*S_wall_office); 
R2 = (w_brick_out/2)/(l_brick*S_wall_office); R3 = R2;
R4 = (w_insulation/4)/(l_insulation*S_wall_office); R6 = R4;
R5 = (w_insulation/2)/(l_insulation*S_wall_office);
R7 = (w_brick_in/4)/(l_brick*S_wall_office); R9 = R7;
R8 = (w_brick_in/2)/(l_brick*S_wall_office);
R10 = 1/(hi*S_wall_office); 

% Glass Wall
R11 = 1/(hi*S_glasswall_office);
R12 = (w_glass/2)/(l_glass*S_glasswall_office); R15 = R12;
R13 = (w_glass/2)/(l_glass*S_glasswall_office) + (w_argon/2)/(l_argon*S_glasswall_office); R14 = R13;
R16 = 1/(ho*S_glasswall_office); 

% Wood door 
R17 = 1/(hi*S_door);
R18 = (w_door/2)/(l_wood*S_door); R19 = R18;
R20 = 1/(ho*S_door);

% Window 
R22 = 1/(hi*S_window); R57 = R22;
R23 = (w_glass/2)/(l_glass*S_window); R26 = R23; R56 = R23; R53 = R26;
R24 = (w_glass/2)/(l_glass*S_window) + (w_argon/2)/(l_argon*S_window);R25 = R24; R55 = R24; R54 = R25;
R27 = 1/(ho*S_window); R52 = R27;

% Partition wall
R28 = 1/(hi*S_plaster); R31 = R28;
R29 = (w_plaster/2)/(l_plaster*S_plaster); R30 = R29;
R21 = (w_plaster/2)/(l_plaster*S_plaster) + (w_air/2)/(l_air*S_plaster); R58 = R21;

% inside door
R32 = 1/(hi*S_door); R35 = R32;
R33 = (w_door/2)/(l_wood*S_door); R34 = 33;

% Outside wall kitchen
R36 = 1/(ho*S_wall_kitchen);
R37 = (w_brick_out/2)/(l_brick*S_wall_kitchen); R38 = R37;
R39 = (w_insulation/4)/(l_insulation*S_wall_kitchen); R41 = R39;
R40 = (w_insulation/2)/(l_insulation*S_wall_kitchen);
R42 = (w_brick_in/4)/(l_brick*S_wall_office); R44 = R42;
R43 = (w_brick_in/2)/(l_brick*S_wall_kitchen);
R45 = 1/(hi*S_wall_kitchen); 

% Glasswall kitchen 
R46 = 1/(ho*S_glasswall_kitchen);
R47 = (w_glass/2)/(l_glass*S_glasswall_kitchen); R50 = R47;
R48 = (w_glass/2)/(l_glass*S_glasswall_kitchen) + (w_argon/2)/(l_argon*S_glasswall_kitchen); R49 = R48;
R51 = 1/(hi*S_glasswall_kitchen);

Rvect = [R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24,R25,R26,R27,R28,R29,R30,R31,R32,R33,R34,R35,R36,R37,R38,R39,R40,R41,R42,R43,R44,R45,R46,R47,R48,R49,R50,R51,R52,R53,R54,R55,R56,R57,R58];
Gvect = [];
for i = (1:58)
    Gvect(i) = 1/Rvect(i);
end
G = diag(Gvect);


### 5.2 Test simulation

#### 5.2.1 TC to SS

Explication Thibaut + expliquer le time step
Matrix f et b


In [ ]:
% matrix f of solar radiation, energy production, and outputs (heat flow sources)
fi = zeros(51,1);
for i = [1 9 10 11 15 16 18 19 23 24 26 27 29 30 31 39 40 44 45 49]
    fi(i) = 1;
end 
yi = zeros(51,1); yi(10) = 1; yi(30) = 1;

% matrix b of constant temperatures sources
We have put a value of temperature for the branchs which are connected to a temperature (outside temperature for example).

b= zeros(58,1);

for i = [1 20 16 27 36 46 52]
    b(i)=1;
end

% TC to SS
[As,Bs,Cs,Ds] = fTC2SS(A,G,b,C,fi,yi);

dtmax=min(-2./eig(As));
dt = 2; %[s]

#### 5.2.2 Simulation with T0 = 1°C

In [ ]:
% simulation with T0 = 1°C, no f

duration = 3600*24 ;   % [s] time duration 
n = floor(duration/dt); % no of time samples

Time = 0:dt:(n-1)*dt;   % time
nth = size(As,1);       % no of state variables
th = zeros(nth,n);      % zero initial conditions
u = zeros(27,n);         % u = [To To To Tsp Phio Phii Qaux Phia]
u(1:7,:) = ones(7,n);   % To = step variation
for k = 1:n-1
 th(:,k+1) = (eye(nth) + dt*As)*th(:,k) + dt*Bs*u(:,k);
end
y = Cs*th + Ds*u;
subplot(3,1,1)
plot(Time/3600,y) 
xlabel('Time [h]'),ylabel('T [C]')
title('Step response for T_o = 1 C'), 

plt.show()

Here are the results obtained that verifies the coherence of the code :

### 5.3 Dynamic Simulation

After we checked the code in the static state, we are runing the code in dynamic.

#### 5.3.1 Solar radiation, energy production and output matrix f

We have put a value for the nodes where there is a source (radiator or electronic devices in a room for example).

In [ ]:
f=zeros(20,period);
E = RadHDif'./3600 ; %+ RadNDir'./3600;
abs_glass = 0.2; tau_glass=0.8;
abs_wall = 0.93;
abs_door = 0.9;
abs_plaster = 0.90;
f(1,:) = E *S_wall_office*abs_wall; %1
f(2,:) = E *abs_wall* S_glasswall_office*tau_glass*(S_wall_office/(S_wall_office+S_plaster+2*S_door+S_window))+...
       E *abs_wall* S_window*tau_glass*(2.5*4.7)/(2.5*4.7+S_plaster+S_door+S_glasswall_office);%9
f(3,:) = E *abs_glass* S_window*tau_glass*S_glasswall_office/(2.5*4.7+S_plaster+S_door+S_glasswall_office); %11
f(4,:) = E *S_glasswall_office*abs_glass ;%15
f(5,:) = E *abs_door* S_glasswall_office*tau_glass*(S_door/(S_wall_office+S_plaster+2*S_door+S_window));%16
f(6,:) = E *abs_door*S_door;%18
f(7,:) = E *abs_glass* S_glasswall_office*tau_glass*(S_window/(S_wall_office+S_plaster+2*S_door+S_window));%19
f(8,:) = E *S_window*abs_glass; %23
f(9,:) =E *abs_plaster*S_glasswall_office*tau_glass*(S_plaster/(S_wall_office+S_plaster+2*S_door+S_window))+...
        E *abs_plaster*S_window*tau_glass*(S_plaster/(4.7*2.5+S_plaster+S_door+S_glasswall_office)); %24
f(10,:) = E *abs_plaster* S_glasswall_kitchen*tau_glass*(S_plaster/(S_wall_kitchen+S_plaster+S_door+S_window))+...
        E *abs_plaster* S_window*tau_glass*(S_plaster/(4.7*2.5+S_plaster+S_door+S_glasswall_kitchen));%26
f(11,:) = E *abs_door* S_glasswall_office*tau_glass*(S_door/(S_wall_office+S_plaster+2*S_door+S_window))+...
        E *abs_door* S_window*tau_glass*(S_door/(4.7*2.5+S_plaster+S_door+S_glasswall_office)); %27
f(12,:) =  E *abs_door* S_glasswall_kitchen*tau_glass*(S_door/(S_wall_kitchen+S_plaster+S_door+S_window))+...
        E *abs_door* S_window*tau_glass*(S_door/(4.7*2.5+S_plaster+S_door+S_glasswall_kitchen));%29
f(13,:) = E *S_wall_kitchen*abs_wall;  %31
f(14,:) =E *abs_wall* S_glasswall_kitchen*tau_glass*(S_wall_kitchen/(S_wall_kitchen+S_plaster+S_door+S_window))+...
       E *abs_wall* S_window*tau_glass*(2.5*4.7)/(2.5*4.7+S_plaster+S_door+S_glasswall_kitchen); %39
f(15,:) = E *S_glasswall_kitchen*abs_glass; %40
f(16,:) = E *abs_glass* S_window*tau_glass*S_glasswall_kitchen/(2.5*4.7+S_plaster+S_door+S_glasswall_kitchen);%44
f(17,:) = E *S_window*abs_glass; %45
f(18,:) = E *abs_glass* S_glasswall_kitchen*tau_glass*(S_window/(S_wall_kitchen+S_plaster+S_door+S_window));%49
f(19,:) = 100; %10 Electronic + occupants + lightings
f(20,:) = 100; %30 Electronic + occupants + lightings

#### 5.3.2 Dynamic simulation

In [ ]:
% interpolate data for time step dt
Temp1 = interp1(Time, Temp, [Time(1):dt:Time(end)]');
ff = zeros(20,length([Time(1):dt:Time(end)]));
for i = 1:20
     
     ff(i,:) = interp1(Time, f(i,:), [Time(1):dt:Time(end)]');
end
Time1 = [Time(1):dt:Time(end)]';

% u implementation

u = [ Temp1 Temp1 Temp1 Temp1 Temp1 Temp1 Temp1 ff'];
u=u';

% Memory alocation and initial value

n = size(Time1,1);
th = zeros(nth,n);


for k = 1:n-1
 th(:,k+1) = (eye(nth) + dt*As)*th(:,k) + dt*Bs*u(:,k);
end
y = Cs*th + Ds*u;
subplot(3,1,2)
freqz(Time1/3600,y,Time1/3600,Temp1)
xlabel('Time [h]'),ylabel('T [C]')
title('Simulation for weather')
     


## 6. 2nd Simulation

In this part, we are going to take in account the orientation of the building, as indicated on the following plan : 

<img src="PLAN_projet.png" alt="drawing" width="800"/>

Initial temperatures will arbitrarily be set to a value located within the comfort temperature range of the season being modelled, thus permitting us to consider a steady-state from the very start of the simulation. For winter simulations, the initial temperature will be set at 16°C. For summer simulations, it will be 25°C.


## 7. Improvements

In this final part, we are going to propose some improvement in order to attain the target temperature in summer and in winter.